<a href="https://colab.research.google.com/github/ByungjunKim/LODinG_OpenAlex/blob/main/LOD_and_OpenAlex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LOD and OpenAlex

In [1]:
pip install -U -q requests natsort tqdm SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00


In [13]:
import pandas as pd
import requests
from tqdm.auto import tqdm
tqdm.pandas()
import json
import glob
import time
import re
from natsort import natsorted
import itertools
import matplotlib.pyplot as plt
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.parse import quote

In [20]:
def request_until_success(url, timeout=20, delay=3):
    """
    Continuously makes a request to the specified URL with a timeout until the request is successful.

    Args:
    - url (str): The URL to request.
    - timeout (int): The timeout for the request in seconds.
    - delay (int): The delay between retries in seconds.

    Returns:
    - response: The successful response from the server.
    """
    attempt = 1
    while True:
        try:
            # print(f"Attempt {attempt}")
            # response = requests.get(url,headers=headers, timeout=timeout)
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()  # Raises HTTPError for bad responses
            # print("Request successful")
            return response.json()
        except requests.Timeout as e:
            print(f"Request timed out: {e}")
        except requests.RequestException as e:
            print(f"Request failed: {e}")

        # print(f"Retrying in {delay} seconds...")
        time.sleep(delay)
        attempt += 1

In [21]:
email = 'kuntakim88@gmail.com' # any email
cursor = '*'

### LOD
Example Query : https://tinyurl.com/2aq7g67x

In [22]:
endpoint_url = "https://query.mimotext.uni-trier.de/proxy/wdqs/bigdata/namespace/wdq/sparql"

query = """PREFIX mmdt: <http://data.mimotext.uni-trier.de/prop/direct/>
PREFIX mmd: <http://data.mimotext.uni-trier.de/entity/>

SELECT ?scholarlyWork ?scholarlyWorkLabel ?zotero_link ?pub_date ?title
WHERE {

?scholarlyWork mmdt:P2 mmd:Q3;
      mmdt:P9 ?pub_date;
      mmdt:P19 ?zotero_link;
      mmdt:P4 ?title.

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

}
"""

In [23]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query)

In [24]:
results["results"]["bindings"][0]

{'scholarlyWork': {'type': 'uri',
  'value': 'http://data.mimotext.uni-trier.de/entity/Q3928'},
 'zotero_link': {'type': 'uri',
  'value': 'https://www.zotero.org/groups/2342956/mimotext/items/QUESHZGA'},
 'title': {'xml:lang': 'de',
  'type': 'literal',
  'value': 'Die französische Aufklärung: Literatur, Gesellschaft und Kultur des 18. Jahrhunderts'},
 'pub_date': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
  'type': 'literal',
  'value': '1998-01-01T00:00:00Z'},
 'scholarlyWorkLabel': {'xml:lang': 'en',
  'type': 'literal',
  'value': 'Bauer-Funke_1998a'}}

In [25]:
def flatten_dict(d):
    return {k: v['value'] for k, v in d.items()}
flattened_data = flattened_data = [flatten_dict(d) for d in results["results"]["bindings"]]
df_lod = pd.DataFrame(flattened_data)
df_lod.head()

,scholarlyWork,zotero_link,title,pub_date,scholarlyWorkLabel
0,http://data.mimotext.uni-trier.de/entity/Q3928,https://www.zotero.org/groups/2342956/mimotext...,"Die französische Aufklärung: Literatur, Gesell...",1998-01-01T00:00:00Z,Bauer-Funke_1998a
1,http://data.mimotext.uni-trier.de/entity/Q3929,https://www.zotero.org/groups/2342956/mimotext...,Französische Literatur in Einzeldarstellungen,1981-01-01T00:00:00Z,Brockmeier_Wetzel_1981a
2,http://data.mimotext.uni-trier.de/entity/Q3930,https://www.zotero.org/groups/2342956/mimotext...,"Handbuch Französisch: Sprache, Literatur, Kult...",2008-01-01T00:00:00Z,Kolboom_ua_2008a
3,http://data.mimotext.uni-trier.de/entity/Q3931,https://www.zotero.org/groups/2342956/mimotext...,18. Jahrhundert: Roman,2006-01-01T00:00:00Z,Rieger_Krauß_2006a
4,http://data.mimotext.uni-trier.de/entity/Q3932,https://www.zotero.org/groups/2342956/mimotext...,Französische Aufklärung. Bürgerliche Emanzipat...,1979-01-01T00:00:00Z,Schröder_1979a


In [26]:
# string to_datetime
df_lod['pub_date'] = pd.to_datetime(df_lod['pub_date'])

### LOD to OpenAlex API
using time and publication_year

In [27]:
test_title = df_lod['title'][1]
test_title

'Französische Literatur in Einzeldarstellungen'

In [28]:
test_year = df_lod['pub_date'][1].year
test_year

1981

In [29]:
# url with a placeholder for cursor
# example_url_with_cursor = f'https://api.openalex.org/works?filter=title_and_abstract.search:japan,publication_year:2000&per-page=200&cursor={cursor}&mailto={email}'
example_url_with_cursor = f'https://api.openalex.org/works?&filter=display_name.search:{test_title},publication_year:{test_year}&per-page=200&cursor={cursor}&mailto={email}'

In [30]:
page_with_results = requests.get(example_url_with_cursor).json()

In [31]:
# metadata
page_with_results['meta']

{'count': 2,
 'db_response_time_ms': 75,
 'page': None,
 'per_page': 200,
 'next_cursor': 'Ils0OC4xNTM2OTQsIDM0NzE1NTIwMDAwMCwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cyMjM5NjY1NzkyJ10i',
 'groups_count': None}

In [32]:
# results
page_with_results['results'][0]

{'id': 'https://openalex.org/W2524711099',
 'doi': 'https://doi.org/10.1007/978-3-476-03150-1',
 'title': 'Französische Literatur in Einzeldarstellungen',
 'display_name': 'Französische Literatur in Einzeldarstellungen',
 'relevance_score': 258.46857,
 'publication_year': 1981,
 'publication_date': '1981-01-01',
 'ids': {'openalex': 'https://openalex.org/W2524711099',
  'doi': 'https://doi.org/10.1007/978-3-476-03150-1',
  'mag': '2524711099'},
 'language': 'de',
 'primary_location': {'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1007/978-3-476-03150-1',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S4306463491',
   'display_name': 'J.B. Metzler eBooks',
   'issn_l': None,
   'issn': None,
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P4310319879',
   'host_organization_name': 'J.B. Metzler',
   'host_organization_lineage': ['https://openalex.org/P4310319879',
    'https://openalex.org/P4310319965'],
   'host_organiza

In [33]:
# Remove punctuation from the 'title' column
df_lod['title'] = df_lod['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [34]:
# Initialize a new list to store the responses
responses = []

for i in tqdm(range(len(df_lod))):
    title = df_lod['title'][i]
    # title = quote(title)  # encode the title string to be URL safe
    # title = title.replace('"', '')  # remove internal double quotes
    year = df_lod['pub_date'][i].year
     # Create a range of years from the current year to the next year
    year_range = f"{year-1}-{year+1}"

    url = f'https://api.openalex.org/works?&filter=display_name.search:{title},publication_year:{year_range}&per-page=200&cursor={cursor}&mailto={email}'
    # url = f'https://api.openalex.org/works?&filter=display_name.search:{title},publication_year:{year}&per-page=200&cursor={cursor}&mailto={email}'

    response = requests.get(url).json()
    responses.append(response)

# Add the responses as a new column to the DataFrame
df_lod['openalex'] = pd.Series(responses)

  0%|          | 0/37 [00:00<?, ?it/s]

In [35]:
df_lod['openalex']

0     {'meta': {'count': 0, 'db_response_time_ms': 1...
1     {'meta': {'count': 7, 'db_response_time_ms': 4...
2     {'meta': {'count': 1, 'db_response_time_ms': 1...
3     {'meta': {'count': 4, 'db_response_time_ms': 5...
4     {'meta': {'count': 0, 'db_response_time_ms': 5...
5     {'meta': {'count': 1, 'db_response_time_ms': 6...
6     {'meta': {'count': 1, 'db_response_time_ms': 1...
7     {'meta': {'count': 0, 'db_response_time_ms': 8...
8     {'meta': {'count': 0, 'db_response_time_ms': 1...
9     {'meta': {'count': 0, 'db_response_time_ms': 1...
10    {'meta': {'count': 1, 'db_response_time_ms': 5...
11    {'meta': {'count': 0, 'db_response_time_ms': 1...
12    {'meta': {'count': 0, 'db_response_time_ms': 5...
13    {'meta': {'count': 0, 'db_response_time_ms': 1...
14    {'meta': {'count': 0, 'db_response_time_ms': 8...
15    {'meta': {'count': 0, 'db_response_time_ms': 6...
16    {'meta': {'count': 0, 'db_response_time_ms': 6...
17    {'meta': {'count': 0, 'db_response_time_ms

In [41]:
# If there are matching results from the openalex api (count>0), assign the first result with the highest similarity

df_lod['openalex_matched'] = df_lod['openalex'].map(lambda x:x['results'][0] if x['meta']['count']>0 else pd.NA)
df_lod['openalex_matched']

0                                                  <NA>
1     {'id': 'https://openalex.org/W2524711099', 'do...
2     {'id': 'https://openalex.org/W4239359467', 'do...
3     {'id': 'https://openalex.org/W2067883549', 'do...
4                                                  <NA>
5     {'id': 'https://openalex.org/W4206458875', 'do...
6     {'id': 'https://openalex.org/W1975366277', 'do...
7                                                  <NA>
8                                                  <NA>
9                                                  <NA>
10    {'id': 'https://openalex.org/W2529289951', 'do...
11                                                 <NA>
12                                                 <NA>
13                                                 <NA>
14                                                 <NA>
15                                                 <NA>
16                                                 <NA>
17                                              

In [44]:
# make openalex id(url) column
df_lod['openalex_id'] = df_lod['openalex_matched'].dropna().map(lambda x:x['id'])
df_lod['openalex_id']

0                                  NaN
1     https://openalex.org/W2524711099
2     https://openalex.org/W4239359467
3     https://openalex.org/W2067883549
4                                  NaN
5     https://openalex.org/W4206458875
6     https://openalex.org/W1975366277
7                                  NaN
8                                  NaN
9                                  NaN
10    https://openalex.org/W2529289951
11                                 NaN
12                                 NaN
13                                 NaN
14                                 NaN
15                                 NaN
16                                 NaN
17                                 NaN
18                                 NaN
19                                 NaN
20                                 NaN
21                                 NaN
22                                 NaN
23    https://openalex.org/W2530707955
24                                 NaN
25                       